In [ ]:
import pandas as pd
import json

# Opening JSON file
f = open('Client_data.json')

# Load Data

In [ ]:
data = json.load(f)
df=pd.DataFrame(data)

In [ ]:
df

In [ ]:
type(df['_source'][0])

In [ ]:
from pandas.io.json import json_normalize
df2 = json_normalize(df._source)
df2.head()

In [ ]:
df2.isnull().sum()/len(df2)

In [ ]:
df2.shape

In [ ]:
df2['company'].nunique()

In [ ]:
df2['complaint_what_happened'][1]

In [ ]:
df2['issue'][1]

# Data Preprocessing

In [ ]:
import nltk

In [ ]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 

In [ ]:

def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)




In [ ]:
df2.head(30)

In [ ]:
df2['complaint_what_happened'][1]

In [ ]:
df2['product']=df2['product'].apply(lambda s:preprocess(s))

In [ ]:
df2['complaint_what_happened']=df2['complaint_what_happened'].apply(lambda s:preprocess(s))

In [ ]:
df2['sub_product']=df2['sub_product'].apply(lambda s:preprocess(s))

In [ ]:
from nltk import word_tokenize

In [ ]:
word_data = []
df2["all_text"] = df2["product"]+df2["sub_product"]+df2["complaint_what_happened"]
word_data = df2["all_text"].apply(lambda x:word_tokenize(x))

In [ ]:
word_data

# Topic Modelling

In [ ]:
import gensim

In [ ]:
from gensim.corpora import Dictionary

In [ ]:
dictionary = gensim.corpora.Dictionary(word_data)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in word_data]

In [ ]:
#Bag of Words corpus for our sample document that is  (token_id, token_count)

In [ ]:
corpus[:15]

In [ ]:
from gensim.models import LdaModel

In [ ]:
model = LdaModel(corpus=corpus,id2word=dictionary,num_topics=5,passes=50,chunksize=1500,iterations=200,alpha="auto")

In [ ]:
model.save("lda_model_1")

In [ ]:
model.show_topics(num_topics=5)

In [ ]:
model.log_perplexity(corpus)

In [ ]:
model.get_document_topics(corpus[0])

In [ ]:
def extract(x):
    max_tup = (0,0)
    for i in x:
        if i[1]>max_tup[1]:
            max_tup = i
    return max_tup[0]

In [ ]:
topics = []
for i in corpus:
    topics.append(extract(model.get_document_topics(i)) + 1)

In [ ]:
df_final = pd.DataFrame(["all_text","type"]).T

In [ ]:
pd.set_option('display.max_colwidth', 500)

In [ ]:
df_results.sample(40)

# create model

In [ ]:
X = df_final.all_text
y = df_final.type

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify = y, test_size = 0.2)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [ ]:
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)
X_train_dtm

In [ ]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [ ]:
nb.fit(X_train_dtm, y_train)

In [ ]:
y_pred_class = nb.predict(X_test_dtm)

In [ ]:
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)